In [1]:
import numpy as np
import pandas as pd
from itertools import chain
from model.SVD_Label import SVD_Label
from model.NMF_Label import NMF_Label

# Setting 

In [2]:
FD_DATA = './data/'
FD_WEIGHT = './weight/'
FN_DATA = FD_DATA + 'data_sample.csv'
# FN_WEIGHT = FD_WEIGHT + 'weight_SVD.p'
FN_WEIGHT = FD_WEIGHT + 'weight_NMF.p'

# Load Data

In [29]:
df = pd.read_csv(FN_DATA, encoding='latin-1')
N = df.shape[0]

# Map label to label description 
map_l2d = dict(df.groupby(by='LABEL')['LABEL_DESC'].first().reset_index().values.tolist())

df.columns, N

(Index(['DESC', 'LABEL', 'LABEL_DESC'], dtype='object'), 9327)

# Train SVD Labeller

In [5]:
# Create model
model = SVD_Label(n_components=22)

# Build
universe_desc = set(chain.from_iterable(df.DESC.str.split().values))
universe_label = set(df.LABEL.values)
model.build(universe_x=universe_desc, universe_y=universe_label)

# Compile
model.compile()

# Fit
x = df.DESC.str.split().values
y_true = df.LABEL.values
model.fit(x, y_true, verbose=True)

# Save
model.save_model(FN_WEIGHT)

# Train NMF Labeller

In [27]:
# Create model
model = NMF_Label(n_components=78)

# Build
universe_desc = set(chain.from_iterable(df.DESC.str.split().values))
universe_label = set(df.LABEL.values)
model.build(universe_x=universe_desc, universe_y=universe_label)

# Compile
model.compile()

# Fit
x = df.DESC.str.split().values
y_true = df.LABEL.values
model.fit(x, y_true, verbose=True)

# Save
model.save_model(FN_WEIGHT)

Recon error: 12.689146348762879, Raw matrix norm: 1382.1798881565035


# Validate

In [28]:
# Load
model.load_model(FN_WEIGHT)

# Inference
y_pred = model.predict(x)

# Evaluate
perc_match = (y_pred == y_true).sum() / N
perc_match

0.7798863514527715

In [26]:
label_pred = np.asarray([ map_l2d[x] for x in y_pred ])

df_result = df.copy()
df_result['LABEL_DESC_INFER'] = label_pred

In [14]:
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)
# pd.set_option('display.max_colwidth', -1)

# df_result[df_result['LABEL_DESC'] != df_result['LABEL_DESC_INFER']]